# **Домашнее задание №6**
Рассмотренная последней модель выбора карьеры не предусматривает возможность быть безработным.

Агент считается занятым и выбирающим работу и карьеру.

Также выше была рассмотрена модель поиска работы с сепарацией, в которой агент мог быть уволен с некоторой вероятностью.

1. Объедините эти две модели, то есть модифицируте модель выбора карьеры, добавив в нее возможность быть уволенным.

2. Выпишите все формулы модели; определите резервную заработную плату или же выпишите условие, которому она должна удовлетворять.

3. Напишите код для поиска оптимального решения модели.

4. Постройте комбинированные графики для решений работающего агента и безработного.

    4.1 Найдите оптимальные решения для агента в обоих случаях.

    4.2 Воспользуйтесь функцией contour для отображения решений одного из агентов поверх решений другого.

    4.3 Постройте такие графики для различных значений параметров, например, пособия по безработице.

    4.4 Объясните полученные результаты.

Задание со звездочкой*

Сделать нормальные комментарии к решению!**

**Будет тяжко, но я постараюсь, мне будет тяжело не вставить мемы, но я держусь 

# **Библиотеки**

In [17]:
import time

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.lines import Line2D
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d import Axes3D  # для 3D-графиков

from numba import njit, prange, float64
from numba.experimental import jitclass

from scipy.stats import beta as beta_dist
from scipy.stats import betabinom


# **Пункт 1**

## **1. Объединение моделей**

### **1.1. Описание объединённой модели**

Объединяются две модели из семинара 13–14 (нагло тырю):

- **Модель выбора карьеры** (*Career Worker Problem*).  
  Агент всегда занят и выбирает:

  1. Остаться на текущей работе
  2. Сменить работу (с сохранением карьеры)
  3. Сменить карьеру (обновляются и $\theta$, и $\varepsilon$)

- **Модель поиска работы с сепарацией**.  
  Агент может быть безработным, получать пособие $c$, а работающий увольняется с вероятностью $\alpha \in [0,1]$.

В объединённой модели агент может быть:

- **Работающим**, получающим заработок  
  $$
  w = \theta + \varepsilon,
  $$  
  и выбирающим между тремя действиями.

- **Безработным**, получающим пособие $c$ и решающим, принимать ли предложение.


### **1.2. Структура предложения о работе**

Предложение характеризуется двумя независимыми компонентами:

$$
w = \theta + \varepsilon
$$

где

- $\theta \sim F$ — карьерная компонента;  
- $\varepsilon \sim G$ — качество конкретной позиции;  
- $\theta$ и $\varepsilon$ независимы.

**Экономический смысл:**

- $\theta$ отражает долгосрочные перспективы профессии.  
- $\varepsilon$ отражает качество конкретного работодателя/позиции.  
- При смене работы обновляется только $\varepsilon'$.  
- При смене карьеры обновляются оба значения $(\theta', \varepsilon')$.



### **1.3. Параметры модели**

| Параметр               | Обозначение   | Диапазон           | Описание                              |
|------------------------|---------------|--------------------|---------------------------------------|
| Фактор дисконтирования | $\beta$       | $(0,1)$            | Вес будущих периодов                  |
| Вероятность увольнения | $\alpha$      | $[0,1]$            | Экзогенная вероятность сепарации      |
| Пособие по безработице | $c$           | $\mathbb{R}_+$     | Доход безработного                    |
| Распределение карьеры  | $F$           | на $[0,B]$         | Распределение $\theta$                |
| Распределение работы   | $G$           | на $[0,B]$         | Распределение $\varepsilon$           |
| Функция полезности     | $u(\cdot)$    | возрастающая       | Полезность от дохода                  |



### **1.4. Действия агентов**

**Работающий агент** в период $t$ выбирает:

1. **Остаться на текущей работе**  
   оставляет $(\theta, \varepsilon)$ без изменений;

2. **Сменить работу**  
   $$
   \theta_{t+1} = \theta_t, \qquad \varepsilon_{t+1} \sim G;
   $$

3. **Сменить карьеру**  
   $$
   \theta_{t+1} \sim F, \qquad \varepsilon_{t+1} \sim G.
   $$


**Безработный агент** выбирает:

1. **Принять предложение**  
   перейти в состояние «занят» с заданными $(\theta, \varepsilon)$;

2. **Отказаться**  
   остаться безработным и получить пособие $c$

# **Пункт 2**

## **2. Формулы модели**

### **2.1. Функция ценности работающего агента**

Пусть $v(\theta, \varepsilon)$ — **ценность работающего агента** с текущей работой $(\theta, \varepsilon)$

**Уравнение Беллмана:**

$$
v(\theta, \varepsilon) = \max\{I, \, II, \, III\}
$$

где три альтернативы определяются следующим образом:

#### **I. Остаться на текущей работе**

$$
I = u(\theta + \varepsilon) + \beta \left[ (1 - \alpha) v(\theta, \varepsilon) + \alpha d \right]
$$

**Интерпретация:**

- Агент получает текущую полезность $u(\theta + \varepsilon)$ от зарплаты $w = \theta + \varepsilon$
- С вероятностью $1 - \alpha$ **сохраняет работу** → ценность $v(\theta, \varepsilon)$
- С вероятностью $\alpha$ **увольняется** → переходит в безработицу с ценностью $d$


#### **II. Сменить работу, сохранив карьеру**

**Непрерывная версия:**

$$
II = u\left(\theta + \bar{\varepsilon}\right) + \beta \left[ (1 - \alpha) \int v(\theta, \varepsilon') \, G(d\varepsilon') + \alpha d \right]
$$

**Дискретная версия** (используется в численных методах):

$$
II = u\left(\theta + \sum_{k=1}^{n_\varepsilon} \varepsilon_k g_k\right) + \beta \left[ (1 - \alpha) \sum_{k=1}^{n_\varepsilon} v(\theta, \varepsilon_k) g_k + \alpha d \right]
$$

где:

- $\bar{\varepsilon} = \mathbb{E}[\varepsilon'] = \sum_{k} \varepsilon_k g_k$ — математическое ожидание новой работы;
- $g_k = \mathbb{P}(\varepsilon = \varepsilon_k)$ — вероятности из распределения $G$

**Интерпретация:**

- Агент **сохраняет карьерный капитал** $\theta$ (специализацию, навыки)
- Получает **новую работу** $\varepsilon' \sim G$ в той же профессии
  *Пример:* программист меняет компанию, но остаётся программистом


#### **III. Сменить и работу, и карьеру**

**Непрерывная версия:**

$$
III = u\left(\bar{\theta} + \bar{\varepsilon}\right) + \beta \left[ (1 - \alpha) \iint v(\theta', \varepsilon') \, F(d\theta') G(d\varepsilon') + \alpha d \right]
$$

**Дискретная версия:**

$$
III = u\left(\sum_{i=1}^{n_\theta} \theta_i f_i + \sum_{k=1}^{n_\varepsilon} \varepsilon_k g_k\right) + \beta \left[ (1 - \alpha) \sum_{i=1}^{n_\theta} \sum_{k=1}^{n_\varepsilon} v(\theta_i, \varepsilon_k) f_i g_k + \alpha d \right]
$$

где:

- $\bar{\theta} = \mathbb{E}[\theta'] = \sum_{i} \theta_i f_i$ — ожидаемая карьера;
- $f_i = \mathbb{P}(\theta = \theta_i)$ — вероятности из распределения $F$

**Интерпретация:**

- Агент **«обнуляет» накопленный капитал** и начинает с новыми $(\theta', \varepsilon')$.
- *Пример:* юрист → data scientist (полная переквалификация).


### **2.2. Функция ценности безработного агента**

Пусть $h(\theta, \varepsilon)$ — **ценность безработного агента**, получившего предложение $(\theta, \varepsilon)$.

**Уравнение Беллмана:**

$$
h(\theta, \varepsilon) = \max \left\{ v(\theta, \varepsilon), \;\; u(c) + \beta d \right\}
$$

**Интерпретация:**

- **Принять предложение:** перейти в состояние работающего → ценность $v(\theta, \varepsilon)$.
- **Отказаться:** получить пособие $c$ сегодня и ожидаемую ценность безработного завтра $d$.


### **2.3. Ожидаемая ценность безработицы**

Величина $d$ определяется как **математическое ожидание** ценности безработного в следующем периоде:

**Непрерывная версия:**

$$
d = \mathbb{E}_{\theta, \varepsilon} [h(\theta', \varepsilon')] = \iint h(\theta', \varepsilon') \, F(d\theta') G(d\varepsilon')
$$

**Дискретная версия:**

$$
d = \sum_{i=1}^{n_\theta} \sum_{k=1}^{n_\varepsilon} h(\theta_i, \varepsilon_k) f_i g_k
$$

Подставляя определение $h$:

$$
d = \sum_{i=1}^{n_\theta} \sum_{k=1}^{n_\varepsilon} \max \left\{ v(\theta_i, \varepsilon_k), \; u(c) + \beta d \right\} f_i g_k
$$




### **2.4. Резервная заработная плата**

#### **Определение**

**Резервная заработная плата** $\bar{w}$ — это **минимальная зарплата**, при которой безработный агент готов принять предложение

#### **Условие для резервной зарплаты**

Безработный принимает предложение $(\theta, \varepsilon)$ тогда и только тогда, когда:

$$
v(\theta, \varepsilon) \geq u(c) + \beta d
$$

**Резервная зарплата определяется из условия равенства:**

$$
\boxed{v(\bar{\theta}, \bar{\varepsilon}) = u(c) + \beta d}
$$

где $\bar{w} = \bar{\theta} + \bar{\varepsilon}$.

#### **Граница принятия в пространстве $(\theta, \varepsilon)$**

Определим **область принятия предложений**:

$$
\mathcal{A} = \left\{ (\theta, \varepsilon) \in [0,B]^2 \, : \, v(\theta, \varepsilon) \geq u(c) + \beta d \right\}
$$

**Стратегия безработного:**

- Если $(\theta, \varepsilon) \in \mathcal{A}$ → **принять** предложение.
- Если $(\theta, \varepsilon) \notin \mathcal{A}$ → **отказаться** и продолжить поиск.

#### **Упрощённая формула (при линейной полезности)**

Если $u(w) = w$ (линейная полезность) и агент всегда остаётся на работе (действие I), то:

$$
v(\theta, \varepsilon) = \frac{\theta + \varepsilon + \beta \alpha d}{1 - \beta(1 - \alpha)}
$$

Подставляя в условие резервной зарплаты:

$$
\frac{\bar{w} + \beta \alpha d}{1 - \beta(1 - \alpha)} = c + \beta d
$$

Решая относительно $\bar{w}$:

$$
\bar{w} = [c + \beta d][1 - \beta(1 - \alpha)] - \beta \alpha d
$$

$$
\boxed{\bar{w} = c(1 - \beta + \beta\alpha) + \beta d(1 - \beta)}
$$

#### **Общий случай (неявное условие)**

В общем случае (произвольная $u$, три действия) резервная зарплата определяется **неявно**:

$$
\boxed{\max\{I, II, III\}\Big|_{(\theta,\varepsilon)=(\bar{\theta},\bar{\varepsilon})} = u(c) + \beta d}
$$

где $I, II, III$ — функции ценности из раздела 2.1.



### **2.5. Оптимальные стратегии**

#### **Стратегия работающего агента**

Оптимальная стратегия $\sigma: [0,B]^2 \to \{1, 2, 3\}$:

$$
\sigma(\theta, \varepsilon) = 
\begin{cases}
1, & \text{если } I \geq \max\{II, III\} \quad \text{(остаться)} \\
2, & \text{если } II \geq \max\{I, III\} \quad \text{(новая работа)} \\
3, & \text{если } III \geq \max\{I, II\} \quad \text{(новая карьера)}
\end{cases}
$$

**Экономическая интерпретация:**

- $\sigma = 1$: высокие $\theta$ **и** $\varepsilon$ → «не чини то, что не сломано».
- $\sigma = 2$: высокое $\theta$, низкое $\varepsilon$ → «хорошая профессия, плохая компания».
- $\sigma = 3$: низкие $\theta$ **и** $\varepsilon$ → «кардинальная смена жизни».

#### **Стратегия безработного агента**

Оптимальная стратегия $\pi: [0,B]^2 \to \{0, 1\}$:

$$
\pi(\theta, \varepsilon) = 
\begin{cases}
1, & \text{если } v(\theta, \varepsilon) \geq u(c) + \beta d \quad \text{(принять)} \\
0, & \text{если } v(\theta, \varepsilon) < u(c) + \beta d \quad \text{(отказаться)}
\end{cases}
$$

Эквивалентно через индикаторную функцию:

$$
\pi(\theta, \varepsilon) = \mathbb{1}\left\{ (\theta, \varepsilon) \in \mathcal{A} \right\}
$$

### **2.6. Алгоритм решения (Value Function Iteration)**

Система уравнений решается методом **последовательных итераций**

#### **Инициализация**

1. Задать начальные приближения:
   $$
   v^{(0)}(\theta, \varepsilon) = \frac{\theta + \varepsilon}{1 - \beta}, \quad d^{(0)} = \frac{u(c)}{1 - \beta}
   $$
2. Выбрать критерий сходимости $\varepsilon_{\text{tol}} > 0$ (например, $10^{-4}$).

#### **Итерационный процесс**

Для $n = 0, 1, 2, \ldots$ выполнять:

**Шаг 1:** Обновить функцию ценности **работающего**:

$$
v^{(n+1)}(\theta_i, \varepsilon_k) = \max \left\{ I^{(n)}, \, II^{(n)}, \, III^{(n)} \right\}
$$

где $I^{(n)}, II^{(n)}, III^{(n)}$ вычисляются по формулам из раздела 2.1 с подстановкой $v^{(n)}$ и $d^{(n)}$

**Шаг 2:** Обновить функцию ценности **безработного**:

$$
h^{(n+1)}(\theta_i, \varepsilon_k) = \max \left\{ v^{(n+1)}(\theta_i, \varepsilon_k), \;\; u(c) + \beta d^{(n)} \right\}
$$

**Шаг 3:** Обновить ожидаемую ценность безработицы:

$$
d^{(n+1)} = \sum_{i=1}^{n_\theta} \sum_{k=1}^{n_\varepsilon} h^{(n+1)}(\theta_i, \varepsilon_k) f_i g_k
$$

**Шаг 4:** Проверить сходимость:

$$
\text{error} = \max \left\{ \max_{i,k} |v^{(n+1)}_{i,k} - v^{(n)}_{i,k}|, \;\; |d^{(n+1)} - d^{(n)}| \right\}
$$

- Если $\text{error} < \varepsilon_{\text{tol}}$, то **СТОП**.
- Иначе $n \leftarrow n+1$ и перейти к Шагу 1.

#### **Выход**

После сходимости получаем:

- $v^* = v^{(n+1)}$ — оптимальная функция ценности работающего;
- $h^* = h^{(n+1)}$ — оптимальная функция ценности безработного;
- $d^* = d^{(n+1)}$ — оптимальная ценность безработицы;
- Оптимальные стратегии $\sigma^*$ и $\pi^*$ извлекаются из $v^*$ и $h^*$ по формулам из раздела 2.5.


### **2.7. Сводная таблица формул**

| Объект | Формула | Комментарий |
|--------|---------|-------------|
| **Ценность работающего** | $v(\theta, \varepsilon) = \max\{I, II, III\}$ | Три действия: остаться / работа / карьера |
| **Действие I** | $I = u(\theta + \varepsilon) + \beta[(1-\alpha)v + \alpha d]$ | Остаться на месте |
| **Действие II** | $II = u(\theta + \bar{\varepsilon}) + \beta[(1-\alpha)\bar{v}_\theta + \alpha d]$ | Сменить работу |
| **Действие III** | $III = u(\bar{\theta} + \bar{\varepsilon}) + \beta[(1-\alpha)\bar{v} + \alpha d]$ | Сменить карьеру |
| **Ценность безработного** | $h(\theta, \varepsilon) = \max\{v(\theta,\varepsilon), u(c) + \beta d\}$ | Принять или отказаться |
| **Ожидаемая ценность** | $d = \sum_{i,k} h(\theta_i, \varepsilon_k) f_i g_k$ | Мат. ожидание по предложениям |
| **Резервная зарплата** | $v(\bar{\theta}, \bar{\varepsilon}) = u(c) + \beta d$ | Граница принятия |
| **Стратегия работающего** | $\sigma(\theta,\varepsilon) \in \{1,2,3\}$ | $\arg\max\{I, II, III\}$ |
| **Стратегия безработного** | $\pi(\theta,\varepsilon) = \mathbb{1}\{v \geq u(c) + \beta d\}$ | Принять (1) или отказать (0) |

**Обозначения:**

- $\bar{\varepsilon} = \sum_k \varepsilon_k g_k$ — среднее $\varepsilon$;
- $\bar{\theta} = \sum_i \theta_i f_i$ — среднее $\theta$;
- $\bar{v}_\theta = \sum_k v(\theta, \varepsilon_k) g_k$ — условное ожидание при фиксированном $\theta$;
- $\bar{v} = \sum_{i,k} v(\theta_i, \varepsilon_k) f_i g_k$ — полное ожидание


### **2.8. Ключевые свойства решения**

1. **Монотонность:**  
   $v(\theta, \varepsilon)$ возрастает по обеим компонентам:
   $$
   \theta_1 > \theta_2 \implies v(\theta_1, \varepsilon) \geq v(\theta_2, \varepsilon)
   $$

2. **Граница принятия:**  
   Множество $\mathcal{A}$ имеет вид «сверху-справа»:
   $$
   (\theta, \varepsilon) \in \mathcal{A} \text{ и } (\theta', \varepsilon') \geq (\theta, \varepsilon) \implies (\theta', \varepsilon') \in \mathcal{A}
   $$

3. **Влияние параметров:**

   - $\uparrow c \implies \uparrow \bar{w}$ — щедрое пособие → выше требования;
   - $\uparrow \alpha \implies \downarrow \bar{w}$ — высокий риск увольнения → меньше разборчивость;
   - $\uparrow \beta \implies \uparrow \bar{w}$ — терпеливость → готовность ждать.




# **Пункт 3**

 КОД ДЛЯ ПОИСКА ОПТИМАЛЬНОГО РЕШЕНИЯ

In [18]:
plt.rcParams['figure.figsize'] = (12, 8)


# ЧАСТЬ 1: ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ
def create_beta_binomial_probs(n, a, b):
    """
    Создает дискретное распределение на основе Beta-распределения.
    Аналог BetaBinomial из quantecon через scipy.
    """
    x = np.linspace(0, 1, n + 1)
    probs = beta_dist.pdf(x, a, b)
    return probs / probs.sum()

# ЧАСТЬ 2: КЛАСС МОДЕЛИ

career_data = [
    ('B', float64),           # верхняя граница θ и ε
    ('beta', float64),        # коэффициент дисконтирования
    ('alpha', float64),       # вероятность увольнения
    ('c', float64),           # пособие по безработице
    ('gridsize', float64),    # размер сетки
    ('theta', float64[:]),    # сетка способностей
    ('epsilon', float64[:]),  # сетка соответствия
    ('Fprobs', float64[:]),   # вероятности для θ
    ('Gprobs', float64[:]),   # вероятности для ε
    ('Fmean', float64),       # E[θ]
    ('Gmean', float64),       # E[ε]
]

@jitclass(career_data)
class CareerWorkerWithUnemployment:
    """
    Объединенная модель выбора карьеры с возможностью увольнения
    
    Состояния:
    - Работающий: (θ, ε) где θ - способности, ε - соответствие
    - Безработный: получает пособие c
    
    Решения работающего:
    1. Остаться на текущей работе
    2. Сменить работу в той же карьере  
    3. Начать новую карьеру
    
    Риск: с вероятностью α агент увольняется
    """
    
    def __init__(self, B, beta, alpha, c, gridsize, Fprobs_input, Gprobs_input):
        self.B = B
        self.beta = beta
        self.alpha = alpha
        self.c = c
        self.gridsize = gridsize
        
        # Создаем сетки
        self.theta = np.linspace(0, B, int(gridsize))
        self.epsilon = np.linspace(0, B, int(gridsize))
        
        # Устанавливаем распределения
        self.Fprobs = Fprobs_input
        self.Gprobs = Gprobs_input
        
        # Вычисляем ожидания
        self.Fmean = np.sum(self.theta * self.Fprobs)
        self.Gmean = np.sum(self.epsilon * self.Gprobs)

def create_model(B=5.0, beta=0.95, alpha=0.1, c=2.0, gridsize=50, 
                 Fa=1, Fb=1, Ga=1, Gb=1):
    """Создает модель с заданными параметрами."""
    Fprobs = create_beta_binomial_probs(gridsize - 1, Fa, Fb)
    Gprobs = create_beta_binomial_probs(gridsize - 1, Ga, Gb)
    return CareerWorkerWithUnemployment(B, beta, alpha, c, gridsize, Fprobs, Gprobs)


# ЧАСТЬ 3: ОПЕРАТОРЫ БЕЛЛМАНА 

def create_operators(cw, parallel_flag=True):
    """
    Создает операторы для итерации по функциям ценности
    ИСПРАВЛЕНИЕ: убраны генераторы списков, несовместимые с Numba
    """
    theta, epsilon, beta = cw.theta, cw.epsilon, cw.beta
    alpha, c = cw.alpha, cw.c
    Fprobs, Gprobs = cw.Fprobs, cw.Gprobs
    Fmean, Gmean = cw.Fmean, cw.Gmean
    
    @njit(parallel=parallel_flag)
    def bellman_operator(V, U):
        """
        Оператор Беллмана для работающего: V(θ,ε) = max{V₁, V₂, V₃}
        """
        V_new = np.empty_like(V)
        
        for i in prange(len(V)):
            for j in prange(len(V[i])):
                # Опция 1: остаться на месте
                current_wage = theta[i] + epsilon[j]
                future_value_1 = (1 - alpha) * V[i, j] + alpha * U
                V1 = current_wage + beta * future_value_1
                
                # Опция 2: сменить работу (сохранить θ, новое ε)
                current_wage_2 = theta[i] + Gmean
                future_value_2 = 0.0
                for k in range(len(epsilon)):
                    future_value_2 += ((1 - alpha) * V[i, k] + alpha * U) * Gprobs[k]
                V2 = current_wage_2 + beta * future_value_2
                
                # Опция 3: новая карьера (новые θ и ε)
                current_wage_3 = Fmean + Gmean
                future_value_3 = 0.0
                for ii in range(len(theta)):
                    for jj in range(len(epsilon)):
                        future_value_3 += ((1 - alpha) * V[ii, jj] + alpha * U) * \
                                         Fprobs[ii] * Gprobs[jj]
                V3 = current_wage_3 + beta * future_value_3
                
                V_new[i, j] = max(V1, V2, V3)
        
        return V_new
    
    @njit
    def unemployment_operator(V, U_old):
        """Оператор для безработного: U = c + β·E[max{V, U}]"""
        expected_value = 0.0
        for i in range(len(theta)):
            for j in range(len(epsilon)):
                expected_value += max(V[i, j], U_old) * Fprobs[i] * Gprobs[j]
        
        U_new = c + beta * expected_value
        return U_new
    
    @njit(parallel=parallel_flag)
    def get_greedy_policy(V, U):
        """
        Извлекает оптимальные политики.
        ИСПРАВЛЕНИЕ: явные циклы вместо генераторов списков.
        """
        policy = np.empty(V.shape)
        accept_policy = np.empty(V.shape)
        
        for i in prange(len(V)):
            for j in prange(len(V[i])):
                # Вычисляем V1
                V1 = theta[i] + epsilon[j] + beta * ((1 - alpha) * V[i, j] + alpha * U)
                
                # Вычисляем V2
                V2_wage = theta[i] + Gmean
                V2_future = 0.0
                for k in range(len(epsilon)):
                    V2_future += ((1 - alpha) * V[i, k] + alpha * U) * Gprobs[k]
                V2 = V2_wage + beta * V2_future
                
                # Вычисляем V3
                V3_wage = Fmean + Gmean
                V3_future = 0.0
                for ii in range(len(theta)):
                    for jj in range(len(epsilon)):
                        V3_future += ((1 - alpha) * V[ii, jj] + alpha * U) * \
                                    Fprobs[ii] * Gprobs[jj]
                V3 = V3_wage + beta * V3_future
                
                # Определяем оптимальное действие
                if V1 >= max(V2, V3):
                    policy[i, j] = 1
                elif V2 >= max(V1, V3):
                    policy[i, j] = 2
                else:
                    policy[i, j] = 3
                
                # Решение безработного
                accept_policy[i, j] = 1 if V[i, j] >= U else 0
        
        return policy, accept_policy
    
    return bellman_operator, unemployment_operator, get_greedy_policy


# ЧАСТЬ 4: РЕШЕНИЕ МОДЕЛИ

def solve_model(cw, use_parallel=True, tol=1e-4, maxiter=1000, 
                verbose=True, print_skip=25):
    """
    Решает модель методом итерации по функции ценности.
    
    Алгоритм:
    1. Начальные приближения V₀, U₀
    2. Итерация: V_{n+1} = T[V_n, U_n], U_{n+1} = T_U[V_{n+1}, U_n]
    3. Останов при ||V_{n+1} - V_n|| < tol
    """
    bellman_op, unemployment_op, get_greedy = create_operators(cw, parallel_flag=use_parallel)
    
    # Начальные приближения
    V = np.full((int(cw.gridsize), int(cw.gridsize)), 
                (cw.Fmean + cw.Gmean) / (1 - cw.beta))
    U = cw.c / (1 - cw.beta)
    
    error = np.inf
    iteration = 0
    
    while iteration < maxiter and error > tol:
        V_new = bellman_op(V, U)
        U_new = unemployment_op(V_new, U)
        
        error_V = np.max(np.abs(V - V_new))
        error_U = np.abs(U - U_new)
        error = max(error_V, error_U)
        
        if verbose and iteration % print_skip == 0:
            print(f"Итерация {iteration:3d}: ошибка = {error:.6f}, U = {U_new:.4f}")
        
        V, U = V_new, U_new
        iteration += 1
    
    if error > tol:
        print(f"⚠ Не сошлось! Ошибка = {error:.6f}")
    elif verbose:
        print(f"✓ Сошлось за {iteration} итераций")
    
    policy, accept_policy = get_greedy(V, U)
    return V, U, policy, accept_policy


# ЗАПУСК И ВИЗУАЛИЗАЦИЯ

if __name__ == "__main__":
    # Создаем модель
    cw = create_model(B=5.0, beta=0.95, alpha=0.1, c=2.0, gridsize=50)
    
    # Решаем
    V, U, policy, accept_policy = solve_model(cw, verbose=True, print_skip=50)
    
    # Результаты
    print(f"\n✓ U = {U:.4f}, E[V] = {np.mean(V):.4f}")


Итерация   0: ошибка = 52.594617, U = 92.5946
Итерация  50: ошибка = 0.160965, U = 130.9432
Итерация 100: ошибка = 0.009604, U = 133.4158
Итерация 150: ошибка = 0.000575, U = 133.5639
✓ Сошлось за 183 итераций

✓ U = 133.5718, E[V] = 138.4966


# **Принцип**

Код реализует модель поиска работы МакКолла с увольнением и выбором карьеры методом динамического программирования. 

Работающий агент может остаться на месте, сменить работу или начать новую карьеру, при этом с вероятностью α он увольняется и становится безработным. 

Модель решается через итерации оператора Беллмана с использованием Numba для ускорения: параллельно обновляются функция ценности работающего V(θ,ε) и ценность безработного U, пока не достигнута сходимость.

На выходе получаются оптимальные стратегии (остаться/сменить работу/сменить карьеру) и условия принятия предложений о работе.

Код использует  @jitclass  для эффективного хранения параметров модели и  @njit(parallel=True)  для параллельных вычислений на сетке состояний.

# **4.1** 

ОПТИМАЛЬНЫЕ РЕШЕНИЯ ДЛЯ ОБОИХ АГЕНТОВ

In [19]:
# Решаем модель с базовыми параметрами
print("\nРешение модели с параметрами: B=5.0, β=0.95, α=0.1, c=2.0...")
cw = create_model(B=5.0, beta=0.95, alpha=0.1, c=2.0, gridsize=50)
V, U, policy, accept_policy = solve_model(cw, verbose=True)


# АНАЛИЗ РАБОТАЮЩЕГО АГЕНТА

# Подсчитываем распределение решений
policy_counts = {1: 0, 2: 0, 3: 0}
for i in range(int(cw.gridsize)):
    for j in range(int(cw.gridsize)):
        policy_counts[int(policy[i, j])] += 1

total = sum(policy_counts.values())

print("\n Оптимальная политика π(θ, ε) имеет 3 варианта:")
print("   1 = Остаться на текущей работе (сохранить θ и ε)")
print("   2 = Сменить работу в той же карьере (сохранить θ, получить новое ε)")
print("   3 = Начать новую карьеру (получить новые θ и ε)")

print("\n  Распределение решений:")
print(f"   • Остаться (1):      {policy_counts[1]:4d} состояний ({100*policy_counts[1]/total:5.1f}%)")
print(f"   • Сменить работу (2): {policy_counts[2]:4d} состояний ({100*policy_counts[2]/total:5.1f}%)")
print(f"   • Новая карьера (3):  {policy_counts[3]:4d} состояний ({100*policy_counts[3]/total:5.1f}%)")


# АНАЛИЗ БЕЗРАБОТНОГО АГЕНТА

print("\n Оптимальная политика a(θ, ε) имеет 2 варианта:")
print("   0 = Отказаться от предложения (остаться безработным, получать c)")
print("   1 = Принять предложение (начать работать с зарплатой θ + ε)")

accept_count = int(np.sum(accept_policy))
reject_count = int(cw.gridsize * cw.gridsize - accept_count)

print("\n  Распределение решений:")
print(f"   • Принять (1):    {accept_count:4d} состояний ({100*accept_count/total:5.1f}%)")
print(f"   • Отказаться (0): {reject_count:4d} состояний ({100*reject_count/total:5.1f}%)")

print(f"\n Ценность безработицы: U = {U:.4f}")
print(f"   Средняя ценность работы: E[V] = {np.mean(V):.4f}")
print(f"   Выигрыш от работы: E[V] - U = {np.mean(V) - U:.4f}")

# ВИЗУАЛИЗАЦИЯ
fig = plt.figure(figsize=(16, 12))
theta_grid, epsilon_grid = np.meshgrid(cw.theta, cw.epsilon)

# График 1: Политика работающего
ax1 = fig.add_subplot(2, 2, 1)
levels_policy = [0.5, 1.5, 2.5, 3.5]
contourf1 = ax1.contourf(theta_grid, epsilon_grid, policy.T, 
                         levels=levels_policy, cmap=cm.RdYlGn, alpha=0.8)
contour1 = ax1.contour(theta_grid, epsilon_grid, policy.T, 
                       levels=levels_policy, colors='k', linewidths=2)
ax1.set_xlabel('θ (способности в карьере)', fontsize=12)
ax1.set_ylabel('ε (соответствие работе)', fontsize=12)
ax1.set_title('РАБОТАЮЩИЙ: Оптимальная политика π(θ, ε)', 
              fontsize=13, fontweight='bold')
ax1.text(0.5, 4.2, 'Зона 1:\nОстаться', fontsize=10, ha='center',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.9))
ax1.text(0.5, 2.0, 'Зона 2:\nСменить\nработу', fontsize=10, ha='center',
         bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.9))
ax1.text(3.5, 0.8, 'Зона 3: Новая карьера', fontsize=10, ha='center',
         bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.9))

# График 2: Политика безработного 
ax2 = fig.add_subplot(2, 2, 2)

# Если все принимают или все отказываются, показываем это явно
if reject_count == 0:
    # Все принимают - зеленая заливка
    ax2.fill_between([0, 5], [0, 0], [5, 5], color='lightgreen', alpha=0.7, 
                     label='Принять')
    ax2.text(2.5, 2.5, 'ВСЕ ПРЕДЛОЖЕНИЯ\nПРИНИМАЮТСЯ', 
             fontsize=16, ha='center', va='center', fontweight='bold',
             bbox=dict(boxstyle='round,pad=1', facecolor='white', 
                      edgecolor='green', linewidth=3, alpha=0.9))
    ax2.text(2.5, 4.2, f'Принято: {accept_count} из {total} состояний (100.0%)', 
             fontsize=11, ha='center',
             bbox=dict(boxstyle='round', facecolor='lightgreen', 
                      edgecolor='darkgreen', alpha=0.95))
elif accept_count == 0:
    # Все отказываются - красная заливка
    ax2.fill_between([0, 5], [0, 0], [5, 5], color='lightcoral', alpha=0.7,
                     label='Отказаться')
    ax2.text(2.5, 2.5, 'ВСЕ ПРЕДЛОЖЕНИЯ\nОТКЛОНЯЮТСЯ', 
             fontsize=16, ha='center', va='center', fontweight='bold',
             bbox=dict(boxstyle='round,pad=1', facecolor='white', 
                      edgecolor='red', linewidth=3, alpha=0.9))
else:
    # Смешанная политика - обычная визуализация
    contourf2 = ax2.contourf(theta_grid, epsilon_grid, accept_policy.T,
                             levels=[0, 0.5, 1.5], cmap=cm.RdYlGn, alpha=0.8)
    contour2 = ax2.contour(theta_grid, epsilon_grid, accept_policy.T,
                           levels=[0.5], colors='k', linewidths=3)

# Добавляем линию V = U (резервная зарплата)
cs_reserve = ax2.contour(theta_grid, epsilon_grid, V.T, levels=[U],
                        colors='blue', linewidths=4, linestyles='--', 
                        label=f'V=U (резервная)')
ax2.clabel(cs_reserve, inline=True, fontsize=10, fmt=f'{U:.1f}')

ax2.set_xlabel('θ (способности в карьере)', fontsize=12)
ax2.set_ylabel('ε (соответствие работе)', fontsize=12)
ax2.set_title('БЕЗРАБОТНЫЙ: Решение a(θ, ε)', fontsize=13, fontweight='bold')
ax2.set_xlim([0, 5])
ax2.set_ylim([0, 5])
ax2.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)

# График 3: Функция ценности V(θ, ε)
ax3 = fig.add_subplot(2, 2, 3)
contourf3 = ax3.contourf(theta_grid, epsilon_grid, V.T, levels=20, 
                         cmap=cm.viridis, alpha=0.8)
cbar = plt.colorbar(contourf3, ax=ax3)
cbar.set_label('V(θ, ε)', fontsize=11)
cs3 = ax3.contour(theta_grid, epsilon_grid, V.T, levels=[U],
                  colors='red', linewidths=4, linestyles='--')
ax3.clabel(cs3, inline=True, fontsize=10, fmt=f'{U:.1f}')
ax3.set_xlabel('θ (способности в карьере)', fontsize=12)
ax3.set_ylabel('ε (соответствие работе)', fontsize=12)
ax3.set_title(f'Функция ценности работающего V(θ, ε)\nКрасная линия: U = {U:.2f}', 
              fontsize=13, fontweight='bold')
ax3.grid(True, alpha=0.2, linestyle='--', linewidth=0.5)

# График 4: Выигрыш от работы V - U
ax4 = fig.add_subplot(2, 2, 4)
diff = V - U
contourf4 = ax4.contourf(theta_grid, epsilon_grid, diff.T, levels=20,
                         cmap=cm.plasma, alpha=0.8)
cbar4 = plt.colorbar(contourf4, ax=ax4)
cbar4.set_label('V - U', fontsize=11)
cs4 = ax4.contour(theta_grid, epsilon_grid, diff.T, levels=[0],
                  colors='white', linewidths=4, linestyles='-')
ax4.clabel(cs4, inline=True, fontsize=10, fmt='0')
ax4.set_xlabel('θ (способности в карьере)', fontsize=12)
ax4.set_ylabel('ε (соответствие работе)', fontsize=12)
ax4.set_title('Выигрыш от работы: V(θ, ε) - U\n(белая линия: V = U)', 
              fontsize=13, fontweight='bold')
ax4.grid(True, alpha=0.2, linestyle='--', linewidth=0.5)

plt.tight_layout()
plt.savefig('hw6_point_4_1.png', dpi=150, bbox_inches='tight')
print("\n✓ График сохранен: hw6_point_4_1.png")
plt.show()


Решение модели с параметрами: B=5.0, β=0.95, α=0.1, c=2.0...
Итерация   0: ошибка = 52.594617, U = 92.5946
Итерация  25: ошибка = 0.686143, U = 122.8269
Итерация  50: ошибка = 0.160965, U = 130.9432
Итерация  75: ошибка = 0.039254, U = 132.9297
Итерация 100: ошибка = 0.009604, U = 133.4158
Итерация 125: ошибка = 0.002350, U = 133.5348
Итерация 150: ошибка = 0.000575, U = 133.5639
Итерация 175: ошибка = 0.000141, U = 133.5710
✓ Сошлось за 183 итераций

 Оптимальная политика π(θ, ε) имеет 3 варианта:
   1 = Остаться на текущей работе (сохранить θ и ε)
   2 = Сменить работу в той же карьере (сохранить θ, получить новое ε)
   3 = Начать новую карьеру (получить новые θ и ε)

  Распределение решений:
   • Остаться (1):       315 состояний ( 12.6%)
   • Сменить работу (2):  576 состояний ( 23.0%)
   • Новая карьера (3):  1609 состояний ( 64.4%)

 Оптимальная политика a(θ, ε) имеет 2 варианта:
   0 = Отказаться от предложения (остаться безработным, получать c)
   1 = Принять предложение (нача

/var/folders/tp/h3xwbjzj52g1cjhylcmqhphr0000gn/T/ipykernel_49173/1617115343.py:99: UserWarning: The following kwargs were not used by contour: 'label'
  cs_reserve = ax2.contour(theta_grid, epsilon_grid, V.T, levels=[U],



✓ График сохранен: hw6_point_4_1.png


/var/folders/tp/h3xwbjzj52g1cjhylcmqhphr0000gn/T/ipykernel_49173/1617115343.py:145: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


# **4.2** Комбинированные графики с использованием contour

In [20]:
# Решаем базовую модель
print("\nРешение базовой модели...")
cw = create_model(B=5.0, beta=0.95, alpha=0.1, c=2.0, gridsize=50)
V, U, policy, accept_policy = solve_model(cw, verbose=False)
print(f"✓ U = {U:.4f}, E[V] = {np.mean(V):.4f}")

# Создаем сетку
theta_grid, epsilon_grid = np.meshgrid(cw.theta, cw.epsilon)

# ВИЗУАЛИЗАЦИЯ: 6 КОМБИНИРОВАННЫХ ГРАФИКОВ

fig = plt.figure(figsize=(18, 12))

# График 1: Политика работающего + граница V=U безработного
ax1 = fig.add_subplot(2, 3, 1)
levels_policy = [0.5, 1.5, 2.5, 3.5]
ax1.contourf(theta_grid, epsilon_grid, policy.T,
            levels=levels_policy, cmap=cm.RdYlGn, alpha=0.6)
contour_boundary = ax1.contour(theta_grid, epsilon_grid, V.T, levels=[U],
                               colors='blue', linewidths=4, linestyles='--')
ax1.clabel(contour_boundary, inline=True, fontsize=10, fmt=f'V=U={U:.1f}')
ax1.set_xlabel('θ (способности)', fontsize=11)
ax1.set_ylabel('ε (соответствие)', fontsize=11)
ax1.set_title('НАЛОЖЕНИЕ 1: Политика работающего + граница безработного\n' + 
              '(синяя линия: выше - принять, ниже - отказаться)',
              fontsize=12, fontweight='bold')
ax1.text(0.5, 4.0, '1', fontsize=14, ha='center', weight='bold')
ax1.text(0.5, 2.0, '2', fontsize=14, ha='center', weight='bold')
ax1.text(3.5, 0.8, '3', fontsize=14, ha='center', weight='bold')

# График 2: Решения безработного + контуры политики работающего
ax2 = fig.add_subplot(2, 3, 2)
ax2.contourf(theta_grid, epsilon_grid, accept_policy.T,
            levels=[0, 0.5, 1.5], cmap=cm.coolwarm, alpha=0.6)
contour_policy = ax2.contour(theta_grid, epsilon_grid, policy.T,
                             levels=levels_policy, colors='black', 
                             linewidths=2.5, linestyles='-')
ax2.clabel(contour_policy, inline=True, fontsize=10)
ax2.set_xlabel('θ (способности)', fontsize=11)
ax2.set_ylabel('ε (соответствие)', fontsize=11)
ax2.set_title('НАЛОЖЕНИЕ 2: Решения безработного + границы работающего\n' +
              '(черные линии: 1=остаться, 2=сменить, 3=новая)',
              fontsize=12, fontweight='bold')
ax2.text(2.5, 4.0, 'ПРИНЯТЬ', fontsize=11, ha='center', weight='bold',
         bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))

# График 3: Выигрыш V-U + обе политики
ax3 = fig.add_subplot(2, 3, 3)
diff = V - U
contourf3 = ax3.contourf(theta_grid, epsilon_grid, diff.T, levels=20,
                         cmap=cm.viridis, alpha=0.7)
cbar = plt.colorbar(contourf3, ax=ax3, label='V - U')
ax3.contour(theta_grid, epsilon_grid, policy.T, levels=levels_policy,
           colors='white', linewidths=2, alpha=0.8)
ax3.contour(theta_grid, epsilon_grid, V.T, levels=[U],
           colors='red', linewidths=4, linestyles='--')
ax3.set_xlabel('θ (способности)', fontsize=11)
ax3.set_ylabel('ε (соответствие)', fontsize=11)
ax3.set_title('НАЛОЖЕНИЕ 3: Выигрыш V-U + обе политики\n' +
              '(белые: работающий, красная: V=U)',
              fontsize=12, fontweight='bold')

# График 4: 3D представление
ax4 = fig.add_subplot(2, 3, 4, projection='3d')
surf = ax4.plot_surface(theta_grid, epsilon_grid, V.T, cmap=cm.viridis,
                        alpha=0.6, linewidth=0, antialiased=True)
U_plane = np.full_like(V.T, U)
ax4.plot_surface(theta_grid, epsilon_grid, U_plane, color='red',
                alpha=0.3, linewidth=0)
ax4.set_xlabel('θ', fontsize=10)
ax4.set_ylabel('ε', fontsize=10)
ax4.set_zlabel('V(θ,ε)', fontsize=10)
ax4.set_title('3D: V(θ,ε) и U\n(красная плоскость: U)',
              fontsize=12, fontweight='bold')
ax4.view_init(elev=20, azim=45)

# График 5: Полупрозрачное наложение
ax5 = fig.add_subplot(2, 3, 5)
ax5.contourf(theta_grid, epsilon_grid, policy.T, levels=levels_policy,
            cmap=cm.RdYlGn, alpha=0.4)
ax5.contour(theta_grid, epsilon_grid, accept_policy.T, levels=[0.5],
           colors='blue', linewidths=4, label='Граница принятия')
ax5.contour(theta_grid, epsilon_grid, policy.T, levels=levels_policy,
           colors='black', linewidths=2, alpha=0.7)
ax5.set_xlabel('θ (способности)', fontsize=11)
ax5.set_ylabel('ε (соответствие)', fontsize=11)
ax5.set_title('НАЛОЖЕНИЕ 4: Полупрозрачные слои\n' +
              '(синяя: граница безработного)',
              fontsize=12, fontweight='bold')
legend_elements = [
    Line2D([0], [0], color='black', lw=2, label='Границы работающего'),
    Line2D([0], [0], color='blue', lw=4, label='Граница безработного')
]
ax5.legend(handles=legend_elements, loc='upper left', fontsize=9)

# График 6: Сечение вдоль диагонали
ax6 = fig.add_subplot(2, 3, 6)
n = int(cw.gridsize)
diagonal_theta = cw.theta
diagonal_V = np.array([V[i, i] for i in range(n)])
diagonal_policy = np.array([policy[i, i] for i in range(n)])

ax6_twin = ax6.twinx()
ax6.plot(diagonal_theta, diagonal_V, 'b-', linewidth=3, label='V(θ, θ)')
ax6.axhline(y=U, color='red', linestyle='--', linewidth=3, label=f'U = {U:.2f}')
ax6.set_xlabel('θ = ε (вдоль диагонали)', fontsize=11)
ax6.set_ylabel('Ценность V', fontsize=11, color='b')
ax6.tick_params(axis='y', labelcolor='b')

ax6_twin.plot(diagonal_theta, diagonal_policy, 'go-', linewidth=2, 
              markersize=4, alpha=0.6, label='Политика работающего')
ax6_twin.set_ylabel('Номер действия', fontsize=11, color='g')
ax6_twin.tick_params(axis='y', labelcolor='g')
ax6_twin.set_ylim([0.5, 3.5])

ax6.set_title('СЕЧЕНИЕ: Вдоль диагонали θ = ε\n' +
              '(синяя: V, красная: U, зеленая: политика)',
              fontsize=12, fontweight='bold')
ax6.legend(loc='upper left', fontsize=9)
ax6.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('hw6_point_4_2.png', dpi=150, bbox_inches='tight')
print("\n✓ График сохранен: hw6_point_4_2.png")
plt.show()



Решение базовой модели...
✓ U = 133.5718, E[V] = 138.4966


/var/folders/tp/h3xwbjzj52g1cjhylcmqhphr0000gn/T/ipykernel_49173/2300862717.py:81: UserWarning: The following kwargs were not used by contour: 'label'
  ax5.contour(theta_grid, epsilon_grid, accept_policy.T, levels=[0.5],



✓ График сохранен: hw6_point_4_2.png


/var/folders/tp/h3xwbjzj52g1cjhylcmqhphr0000gn/T/ipykernel_49173/2300862717.py:125: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


# **Принцип**

Код сначала решает базовую динамическую модель поиска работы: строится объект модели  cw , затем с помощью  solve_model  находятся функция ценности работающего  V , 

ценность безработного  U  и оптимальные политики.  На сетке  θ × ε  создаётся двумерная решётка  theta_grid, epsilon_grid  для последующей визуализации.  

Далее рисуется фигура с 6 подграфиками, где с помощью  contourf  и  contour  показываются: политика работающего, область принятия/отказа безработного и уровень, 

где , то есть граница безразличия между работой и безработицей.  Один из графиков — 3D-поверхность функции ценности  V(θ, ε)  и горизонтальная плоскость  U , что иллюстрирует, в каких состояниях работа доминирует над безработицей. 

 В конце всё упаковывается  plt.tight_layout() , сохраняется в файл и выводится на экран

# **4.3** Анализ при различных значениях параметров  

In [21]:
# ОПРЕДЕЛЕНИЕ МОДЕЛИ (CareerWorkerProblem)

class CareerWorkerProblem:
    """
    Модель поиска работы с возможностью увольнения и смены карьеры
    """
    def __init__(self, B=5.0, beta=0.95, alpha=0.1, c=2.0, gridsize=50, Fa=1, Fb=1, Ga=1, Gb=1):
        self.beta, self.gridsize, self.B = beta, gridsize, B
        self.alpha, self.c = alpha, c
        
        # Сетки для θ (способности) и ε (соответствие)
        self.theta = np.linspace(0, B, gridsize)
        self.epsilon = np.linspace(0, B, gridsize)
        
        # Распределения (используем betabinom из scipy)
        self.Fprobs = betabinom.pmf(np.arange(gridsize), gridsize - 1, Fa, Fb)
        self.Gprobs = betabinom.pmf(np.arange(gridsize), gridsize - 1, Ga, Gb)
        
        # Нормализуем
        self.Fprobs = self.Fprobs / self.Fprobs.sum()
        self.Gprobs = self.Gprobs / self.Gprobs.sum()
        
        self.Fmean = np.sum(self.theta * self.Fprobs)
        self.Gmean = np.sum(self.epsilon * self.Gprobs)

def create_model(B=5.0, beta=0.95, alpha=0.1, c=2.0, gridsize=50):
    """Создает экземпляр модели с заданными параметрами"""
    return CareerWorkerProblem(B=B, beta=beta, alpha=alpha, c=c, gridsize=gridsize)


# ОПЕРАТОР БЕЛЛМАНА
def operator_factory(cw, parallelflag=True):
    """Создает оператор Беллмана для итераций"""
    theta, epsilon, beta = cw.theta, cw.epsilon, cw.beta
    Fprobs, Gprobs = cw.Fprobs, cw.Gprobs
    Fmean, Gmean = cw.Fmean, cw.Gmean
    alpha, c = cw.alpha, cw.c
    
    @njit(parallel=parallelflag)
    def Q(v):
        vnew = np.empty_like(v)
        for i in prange(len(v)):
            for j in prange(len(v)):
                # v1: продолжить работать на текущей работе
                v1 = theta[i] + epsilon[j] + beta * ((1 - alpha) * v[i, j] + alpha * c)
                
                # v2: искать новую работу (новое ε)
                sum_v2 = 0.0
                for k in range(len(v)):
                    sum_v2 += v[i, k] * Gprobs[k]
                v2 = theta[i] + Gmean + beta * ((1 - alpha) * sum_v2 + alpha * c)
                
                # v3: начать новую карьеру (новые θ и ε)
                sum_v3 = 0.0
                for k in range(len(v)):
                    for l in range(len(v)):
                        sum_v3 += Fprobs[k] * v[k, l] * Gprobs[l]
                v3 = Gmean + Fmean + beta * ((1 - alpha) * sum_v3 + alpha * c)
                
                vnew[i, j] = max(v1, v2, v3)
        return vnew
    
    @njit(parallel=parallelflag)
    def get_greedy(v):
        """Вычисляет жадную политику"""
        sigma = np.empty(v.shape, dtype=np.int32)
        for i in prange(len(v)):
            for j in prange(len(v)):
                # v1: продолжить работать на текущей работе
                v1 = theta[i] + epsilon[j] + beta * ((1 - alpha) * v[i, j] + alpha * c)
                
                # v2: искать новую работу (новое ε)
                sum_v2 = 0.0
                for k in range(len(v)):
                    sum_v2 += v[i, k] * Gprobs[k]
                v2 = theta[i] + Gmean + beta * ((1 - alpha) * sum_v2 + alpha * c)
                
                # v3: начать новую карьеру (новые θ и ε)
                sum_v3 = 0.0
                for k in range(len(v)):
                    for l in range(len(v)):
                        sum_v3 += Fprobs[k] * v[k, l] * Gprobs[l]
                v3 = Gmean + Fmean + beta * ((1 - alpha) * sum_v3 + alpha * c)
                
                if v1 >= max(v2, v3):
                    action = 1  # остаться
                elif v2 >= max(v1, v3):
                    action = 2  # новая работа
                else:
                    action = 3  # новая карьера
                sigma[i, j] = action
        return sigma
    
    return Q, get_greedy

# РЕШЕНИЕ МОДЕЛИ

def solve_model(cw, use_parallel=True, tol=1e-4, max_iter=1000, verbose=True, print_skip=25):
    """
    Решает модель методом итераций функции ценности
    
    Возвращает:
    - V: функция ценности для работающего
    - U: ценность безработного
    - policy: оптимальная политика
    - accept: матрица принятия предложений (V >= U)
    """
    Q, get_greedy = operator_factory(cw, parallelflag=use_parallel)
    
    # Начальное приближение
    v = np.full((cw.gridsize, cw.gridsize), cw.Fmean + cw.Gmean)
    
    # Итерации
    error = np.inf
    for i in range(max_iter):
        vnew = Q(v)
        error = np.max(np.abs(v - vnew))
        
        if verbose and i % print_skip == 0:
            print(f"   Итерация {i}: ошибка = {error:.6f}")
        
        if error < tol:
            break
        v = vnew
    
    if error >= tol:
        print(" Не сошлось!")
    elif verbose:
        print(f" Сошлось за {i+1} итераций")
    
    # Вычисляем ценность безработного
    U_val = cw.c + cw.beta * np.sum(cw.Fprobs[:, np.newaxis] * v * cw.Gprobs[np.newaxis, :])
    
    # Вычисляем оптимальную политику
    policy = get_greedy(v)
    
    # Матрица принятия: где V >= U
    accept = (v >= U_val)
    
    return v, U_val, policy, accept

# ЧАСТЬ 1: ВЛИЯНИЕ ПОСОБИЯ ПО БЕЗРАБОТИЦЕ c
print("\n Исследуемые значения: c ∈ {1.0, 2.0, 3.0, 4.0}")
print("   Остальные параметры: B=5.0, β=0.95, α=0.1\n")

c_values = [1.0, 2.0, 3.0, 4.0]
results_c = {}

start_time = time.time()
for c_val in c_values:
    cw = create_model(B=5.0, beta=0.95, alpha=0.1, c=c_val, gridsize=50)
    V, U, policy, accept = solve_model(cw, verbose=False)
    results_c[c_val] = {
        'cw': cw, 'V': V, 'U': U, 
        'policy': policy, 'accept': accept
    }
    
    accept_pct = 100 * np.sum(accept) / accept.size
    print(f"   c={c_val:.1f}: U={U:6.2f}, E[V]={np.mean(V):6.2f}, " +
          f"принимают={accept_pct:5.1f}%")

print(f"\n  Время решения: {time.time() - start_time:.1f} сек")


# ЧАСТЬ 2: ВЛИЯНИЕ ВЕРОЯТНОСТИ УВОЛЬНЕНИЯ α
print("\n Исследуемые значения: α ∈ {0.05, 0.10, 0.20, 0.30}")
print("   Остальные параметры: B=5.0, β=0.95, c=2.0\n")

alpha_values = [0.05, 0.10, 0.20, 0.30]
results_alpha = {}

start_time = time.time()
for alpha_val in alpha_values:
    cw = create_model(B=5.0, beta=0.95, alpha=alpha_val, c=2.0, gridsize=50)
    V, U, policy, accept = solve_model(cw, verbose=False)
    results_alpha[alpha_val] = {
        'cw': cw, 'V': V, 'U': U,
        'policy': policy, 'accept': accept
    }
    
    gap = np.mean(V) - U
    print(f"   α={alpha_val:.2f}: U={U:6.2f}, E[V]={np.mean(V):6.2f}, " +
          f"разрыв={gap:5.2f}")

print(f"\n   ✓ Время решения: {time.time() - start_time:.1f} сек")


# ЧАСТЬ 3: ВЛИЯНИЕ ДИСКОНТ-ФАКТОРА β
print("\n📋 Исследуемые значения: β ∈ {0.90, 0.93, 0.95, 0.97}")
print("   Остальные параметры: B=5.0, α=0.1, c=2.0\n")

beta_values = [0.90, 0.93, 0.95, 0.97]
results_beta = {}

start_time = time.time()
for beta_val in beta_values:
    cw = create_model(B=5.0, beta=beta_val, alpha=0.1, c=2.0, gridsize=50)
    V, U, policy, accept = solve_model(cw, verbose=False)
    results_beta[beta_val] = {
        'cw': cw, 'V': V, 'U': U,
        'policy': policy, 'accept': accept
    }
    
    ratio = np.mean(V) / U if U > 0 else 0
    print(f"   β={beta_val:.2f}: U={U:7.2f}, E[V]={np.mean(V):7.2f}, " +
          f"E[V]/U={ratio:.3f}")

print(f"\n   Время решения: {time.time() - start_time:.1f} сек")


# ВИЗУАЛИЗАЦИЯ: 9 ГРАФИКОВ (3x3)
fig = plt.figure(figsize=(20, 14))

# Базовая сетка
cw_base = results_c[2.0]['cw']
theta_grid, epsilon_grid = np.meshgrid(cw_base.theta, cw_base.epsilon)

# Цвета для графиков
colors_c = ['#8B008B', '#0000FF', '#00AA00', '#FF8800']
colors_alpha = ['#8B008B', '#0000FF', '#00AA00', '#FF0000']
colors_beta = ['#8B008B', '#0000FF', '#00AA00', '#FF0000']


# РЯД 1: ВЛИЯНИЕ c (пособием)

# График 1.1: Boundaries V=U для разных c
ax1 = fig.add_subplot(3, 3, 1)
ax1.set_facecolor('#F0F0F0')

legend_elements = []
for idx, c_val in enumerate(c_values):
    V_c = results_c[c_val]['V']
    U_c = results_c[c_val]['U']
    
    cs = ax1.contour(theta_grid, epsilon_grid, V_c.T, levels=[U_c],
                    colors=[colors_c[idx]], linewidths=5, linestyles='-')
    ax1.clabel(cs, inline=True, fontsize=11, fmt=f'{U_c:.0f}')
    
    legend_elements.append(Line2D([0], [0], color=colors_c[idx], linewidth=5,
                                  label=f'c={c_val:.1f}, U={U_c:.1f}'))

ax1.set_xlabel('θ (способности)', fontsize=11)
ax1.set_ylabel('ε (соответствие)', fontsize=11)
ax1.set_title('Резервная зарплата при разных c\n(выше линии: принять)',
              fontsize=12, weight='bold')
ax1.legend(handles=legend_elements, loc='lower right', fontsize=9, 
          framealpha=0.95, edgecolor='black')
ax1.grid(True, alpha=0.4, color='white', linewidth=1.5)
ax1.set_xlim([0, 5])
ax1.set_ylim([0, 5])

# График 1.2: Зависимость U и E[V] от c
ax2 = fig.add_subplot(3, 3, 2)
U_vals_c = [results_c[c]['U'] for c in c_values]
EV_vals_c = [np.mean(results_c[c]['V']) for c in c_values]
ax2.plot(c_values, U_vals_c, 'o-', linewidth=3, markersize=10,
        color='red', label='U (безработный)')
ax2.plot(c_values, EV_vals_c, 's-', linewidth=3, markersize=10,
        color='blue', label='E[V] (работающий)')
ax2.set_xlabel('c (пособие по безработице)', fontsize=11)
ax2.set_ylabel('Ценность', fontsize=11)
ax2.set_title('Ценности vs пособие c', fontsize=12, weight='bold')
ax2.legend(loc='best', fontsize=10)
ax2.grid(True, alpha=0.3)

# График 1.3: Процент принятия по θ для разных c
ax3 = fig.add_subplot(3, 3, 3)
for idx, c_val in enumerate(c_values):
    accept = results_c[c_val]['accept']
    theta_vals = []
    accept_pct = []
    for i in range(0, 50, 3):
        theta_vals.append(cw_base.theta[i])
        accept_pct.append(100 * np.sum(accept[i, :]) / 50)
    ax3.plot(theta_vals, accept_pct, 'o-', linewidth=2, markersize=6,
            color=colors_c[idx], label=f'c={c_val:.1f}')
ax3.set_xlabel('θ (способности)', fontsize=11)
ax3.set_ylabel('% принятых предложений', fontsize=11)
ax3.set_title('Принятие vs способности θ\nпри разных c',
              fontsize=12, weight='bold')
ax3.legend(loc='best', fontsize=9)
ax3.grid(True, alpha=0.3)
ax3.set_ylim([0, 105])


# РЯД 2: ВЛИЯНИЕ α (риск увольнения)

# График 2.1: Границы V=U для разных α
ax4 = fig.add_subplot(3, 3, 4)
ax4.set_facecolor('#F0F0F0')

legend_elements_alpha = []
for idx, alpha_val in enumerate(alpha_values):
    V_a = results_alpha[alpha_val]['V']
    U_a = results_alpha[alpha_val]['U']
    
    cs = ax4.contour(theta_grid, epsilon_grid, V_a.T, levels=[U_a],
                    colors=[colors_alpha[idx]], linewidths=5, linestyles='-')
    ax4.clabel(cs, inline=True, fontsize=11, fmt=f'{U_a:.0f}')
    
    legend_elements_alpha.append(Line2D([0], [0], color=colors_alpha[idx], linewidth=5,
                                       label=f'α={alpha_val:.2f}, U={U_a:.1f}'))

ax4.set_xlabel('θ (способности)', fontsize=11)
ax4.set_ylabel('ε (соответствие)', fontsize=11)
ax4.set_title('Резервная зарплата при разных α\n(вероятность увольнения)',
              fontsize=12, weight='bold')
ax4.legend(handles=legend_elements_alpha, loc='lower right', fontsize=9,
          framealpha=0.95, edgecolor='black')
ax4.grid(True, alpha=0.4, color='white', linewidth=1.5)
ax4.set_xlim([0, 5])
ax4.set_ylim([0, 5])

# График 2.2: Зависимость U и E[V] от α
ax5 = fig.add_subplot(3, 3, 5)
U_vals_alpha = [results_alpha[a]['U'] for a in alpha_values]
EV_vals_alpha = [np.mean(results_alpha[a]['V']) for a in alpha_values]
gap_vals = [EV_vals_alpha[i] - U_vals_alpha[i] for i in range(len(alpha_values))]

ax5_twin = ax5.twinx()
line1 = ax5.plot(alpha_values, U_vals_alpha, 'o-', linewidth=3, markersize=10,
                color='red', label='U (безработный)')
line2 = ax5.plot(alpha_values, EV_vals_alpha, 's-', linewidth=3, markersize=10,
                color='blue', label='E[V] (работающий)')
line3 = ax5_twin.plot(alpha_values, gap_vals, '^--', linewidth=2, markersize=8,
                     color='green', alpha=0.7, label='E[V] - U')

ax5.set_xlabel('α (вероятность увольнения)', fontsize=11)
ax5.set_ylabel('Ценность', fontsize=11, color='black')
ax5_twin.set_ylabel('Разрыв E[V] - U', fontsize=11, color='green')
ax5_twin.tick_params(axis='y', labelcolor='green')
ax5.set_title('Ценности vs риск увольнения α', fontsize=12, weight='bold')

lines = line1 + line2 + line3
labels = [l.get_label() for l in lines]
ax5.legend(lines, labels, loc='best', fontsize=9)
ax5.grid(True, alpha=0.3)

# График 2.3: Сравнение политик работающего при разных α
ax6 = fig.add_subplot(3, 3, 6)
base_policy = results_alpha[0.1]['policy']
ax6.contourf(theta_grid, epsilon_grid, base_policy.T,
            levels=[0.5, 1.5, 2.5, 3.5], cmap=cm.RdYlGn, alpha=0.3)
legend_elements_policy = []
for idx, alpha_val in enumerate(alpha_values):
    if alpha_val == 0.1:
        continue
    pol = results_alpha[alpha_val]['policy']
    ax6.contour(theta_grid, epsilon_grid, pol.T, levels=[1.5, 2.5],
               colors=colors_alpha[idx], linewidths=2.5, alpha=0.8)
    legend_elements_policy.append(Line2D([0], [0], color=colors_alpha[idx], 
                                        linewidth=2.5, label=f'α={alpha_val:.2f}'))
ax6.set_xlabel('θ (способности)', fontsize=11)
ax6.set_ylabel('ε (соответствие)', fontsize=11)
ax6.set_title('Политики работающего\nпри разных α (фон: α=0.1)',
              fontsize=12, weight='bold')
ax6.legend(handles=legend_elements_policy, loc='upper left', fontsize=9)
ax6.grid(True, alpha=0.2)


# РЯД 3: ВЛИЯНИЕ β (дисконт-фактор)


# График 3.1: Границы V=U для разных β
ax7 = fig.add_subplot(3, 3, 7)
ax7.set_facecolor('#F0F0F0')

legend_elements_beta = []
for idx, beta_val in enumerate(beta_values):
    V_b = results_beta[beta_val]['V']
    U_b = results_beta[beta_val]['U']
    
    cs = ax7.contour(theta_grid, epsilon_grid, V_b.T, levels=[U_b],
                    colors=[colors_beta[idx]], linewidths=5, linestyles='-')
    ax7.clabel(cs, inline=True, fontsize=11, fmt=f'{U_b:.0f}')
    
    legend_elements_beta.append(Line2D([0], [0], color=colors_beta[idx], linewidth=5,
                                      label=f'β={beta_val:.2f}, U={U_b:.1f}'))

ax7.set_xlabel('θ (способности)', fontsize=11)
ax7.set_ylabel('ε (соответствие)', fontsize=11)
ax7.set_title('Резервная зарплата при разных β\n(дисконт-фактор)',
              fontsize=12, weight='bold')
ax7.legend(handles=legend_elements_beta, loc='lower right', fontsize=9,
          framealpha=0.95, edgecolor='black')
ax7.grid(True, alpha=0.4, color='white', linewidth=1.5)
ax7.set_xlim([0, 5])
ax7.set_ylim([0, 5])

# График 3.2: Зависимость U и E[V] от β
ax8 = fig.add_subplot(3, 3, 8)
U_vals_beta = [results_beta[b]['U'] for b in beta_values]
EV_vals_beta = [np.mean(results_beta[b]['V']) for b in beta_values]

ax8.plot(beta_values, U_vals_beta, 'o-', linewidth=3, markersize=10,
        color='red', label='U (безработный)')
ax8.plot(beta_values, EV_vals_beta, 's-', linewidth=3, markersize=10,
        color='blue', label='E[V] (работающий)')
ax8.set_xlabel('β (дисконт-фактор)', fontsize=11)
ax8.set_ylabel('Ценность', fontsize=11)
ax8.set_title('Ценности vs терпеливость β', fontsize=12, weight='bold')
ax8.legend(loc='best', fontsize=10)
ax8.grid(True, alpha=0.3)
growth = EV_vals_beta[-1] / EV_vals_beta[0]
ax8.text(0.92, EV_vals_beta[-1] * 0.9, f'Рост в {growth:.1f}x',
        fontsize=10, bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

# График 3.3: Относительная разница
ax9 = fig.add_subplot(3, 3, 9)
rel_diff_c = [(EV_vals_c[i] - U_vals_c[i]) / U_vals_c[i] * 100 
              for i in range(len(c_values))]
rel_diff_alpha = [(EV_vals_alpha[i] - U_vals_alpha[i]) / U_vals_alpha[i] * 100
                  for i in range(len(alpha_values))]
rel_diff_beta = [(EV_vals_beta[i] - U_vals_beta[i]) / U_vals_beta[i] * 100
                 for i in range(len(beta_values))]

x_c_norm = (np.array(c_values) - c_values[0]) / (c_values[-1] - c_values[0])
x_alpha_norm = (np.array(alpha_values) - alpha_values[0]) / (alpha_values[-1] - alpha_values[0])
x_beta_norm = (np.array(beta_values) - beta_values[0]) / (beta_values[-1] - beta_values[0])

ax9.plot(x_c_norm, rel_diff_c, 'o-', linewidth=2, markersize=8, label='c (пособие)')
ax9.plot(x_alpha_norm, rel_diff_alpha, 's-', linewidth=2, markersize=8, label='α (увольнение)')
ax9.plot(x_beta_norm, rel_diff_beta, '^-', linewidth=2, markersize=8, label='β (дисконт)')

ax9.set_xlabel('Нормализованное изменение параметра (0→1)', fontsize=11)
ax9.set_ylabel('(E[V] - U) / U × 100%', fontsize=11)
ax9.set_title('Относительный выигрыш от работы\nпри изменении параметров',
              fontsize=12, weight='bold')
ax9.legend(loc='best', fontsize=10)
ax9.grid(True, alpha=0.3)
ax9.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)

plt.tight_layout()
plt.savefig('hw6_point_4_3.png', dpi=150, bbox_inches='tight')
print("\n✓ График сохранен: hw6_point_4_3.png")
plt.show()


# ЧИСЛОВЫЕ СВОДКИ



print("\n  ВЛИЯНИЕ ПОСОБИЯ c:")
print(f"   При увеличении c: {c_values[0]}→{c_values[-1]}")
print(f"   • U изменяется: {U_vals_c[0]:.2f}→{U_vals_c[-1]:.2f} (Δ={U_vals_c[-1]-U_vals_c[0]:+.2f})")
print(f"   • E[V] изменяется: {EV_vals_c[0]:.2f}→{EV_vals_c[-1]:.2f} (Δ={EV_vals_c[-1]-EV_vals_c[0]:+.2f})")
print(f"   • Эластичность U по c: {(U_vals_c[-1]-U_vals_c[0])/(c_values[-1]-c_values[0]):.2f}")

print("\n  ВЛИЯНИЕ РИСКА УВОЛЬНЕНИЯ α:")
print(f"   При увеличении α: {alpha_values[0]}→{alpha_values[-1]}")
print(f"   • U изменяется: {U_vals_alpha[0]:.2f}→{U_vals_alpha[-1]:.2f} (Δ={U_vals_alpha[-1]-U_vals_alpha[0]:.2f})")
print(f"   • E[V] изменяется: {EV_vals_alpha[0]:.2f}→{EV_vals_alpha[-1]:.2f} (Δ={EV_vals_alpha[-1]-EV_vals_alpha[0]:.2f})")
print(f"   • Разрыв V-U уменьшается: {gap_vals[0]:.2f}→{gap_vals[-1]:.2f}")

print("\n  ВЛИЯНИЕ ДИСКОНТ-ФАКТОРА β:")
print(f"   При увеличении β: {beta_values[0]}→{beta_values[-1]}")
print(f"   • U изменяется: {U_vals_beta[0]:.2f}→{U_vals_beta[-1]:.2f} (Δ={U_vals_beta[-1]-U_vals_beta[0]:+.2f})")
print(f"   • E[V] изменяется: {EV_vals_beta[0]:.2f}→{EV_vals_beta[-1]:.2f} (Δ={EV_vals_beta[-1]-EV_vals_beta[0]:+.2f})")
print(f"   • Рост ценностей: в {U_vals_beta[-1]/U_vals_beta[0]:.2f}x раза")




 Исследуемые значения: c ∈ {1.0, 2.0, 3.0, 4.0}
   Остальные параметры: B=5.0, β=0.95, α=0.1

   c=1.0: U= 50.06, E[V]= 51.64, принимают= 33.6%
   c=2.0: U= 51.68, E[V]= 52.29, принимают= 31.6%
   c=3.0: U= 53.30, E[V]= 52.95, принимают= 28.2%
   c=4.0: U= 54.92, E[V]= 53.60, принимают= 25.6%

  Время решения: 3.7 сек

 Исследуемые значения: α ∈ {0.05, 0.10, 0.20, 0.30}
   Остальные параметры: B=5.0, β=0.95, c=2.0

   α=0.05: U= 77.84, E[V]= 79.83, разрыв= 1.99
   α=0.10: U= 51.68, E[V]= 52.29, разрыв= 0.61
   α=0.20: U= 31.27, E[V]= 30.81, разрыв=-0.46
   α=0.30: U= 22.82, E[V]= 21.91, разрыв=-0.90

   ✓ Время решения: 3.1 сек

📋 Исследуемые значения: β ∈ {0.90, 0.93, 0.95, 0.97}
   Остальные параметры: B=5.0, α=0.1, c=2.0

   β=0.90: U=  36.91, E[V]=  38.79, E[V]/U=1.051
   β=0.93: U=  44.74, E[V]=  45.95, E[V]/U=1.027
   β=0.95: U=  51.68, E[V]=  52.29, E[V]/U=1.012
   β=0.97: U=  60.70, E[V]=  60.52, E[V]/U=0.997

   Время решения: 3.0 сек

✓ График сохранен: hw6_point_4_3.png

  

/var/folders/tp/h3xwbjzj52g1cjhylcmqhphr0000gn/T/ipykernel_49173/260697942.py:436: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


# **Принцип** 

Код строит и решает динамическую модель выбора карьеры с увольнениями и пособием по безработице, где состояние работника описывается парой  — способности и соответствие.

Функция  solve_model  с помощью оператора Беллмана итерационно находит стационарную функцию ценности работы , ценность безработицы , а также оптимальную политику (остаться, сменить работу, сменить карьеру) 

и область принятия предложений. Затем код перебирает значения трех параметров модели — пособие , риск увольнения  и дисконт-фактор  — и для каждого параметра вычисляет , среднее , разрыв  и долю принятых 

предложений. На основе этих результатов строятся 9 графиков (3×3), которые показывают, как меняются границы безразличия , 

оптимальные политики и относительный выигрыш от работы при изменении параметров. В конце выводятся числовые сводки, которые компактно описывают влияние каждого параметра на ценность работы и безработицы.

# **4.4**  Объяснение полученных результатов

In [22]:
# Решаем модели для иллюстрации
print("\nПодготовка данных для объяснений...")

# Базовая модель
cw_base = create_model(B=5.0, beta=0.95, alpha=0.1, c=2.0, gridsize=50)
V_base, U_base, policy_base, accept_base = solve_model(cw_base, verbose=False)

# Разные c
cw_low_c = create_model(B=5.0, beta=0.95, alpha=0.1, c=1.0, gridsize=50)
V_low_c, U_low_c, policy_low_c, accept_low_c = solve_model(cw_low_c, verbose=False)

cw_high_c = create_model(B=5.0, beta=0.95, alpha=0.1, c=4.0, gridsize=50)
V_high_c, U_high_c, policy_high_c, accept_high_c = solve_model(cw_high_c, verbose=False)

# Разные α
cw_low_alpha = create_model(B=5.0, beta=0.95, alpha=0.05, c=2.0, gridsize=50)
V_low_alpha, U_low_alpha, policy_low_alpha, accept_low_alpha = solve_model(cw_low_alpha, verbose=False)

cw_high_alpha = create_model(B=5.0, beta=0.95, alpha=0.3, c=2.0, gridsize=50)
V_high_alpha, U_high_alpha, policy_high_alpha, accept_high_alpha = solve_model(cw_high_alpha, verbose=False)

print("✓ Все модели решены\n")


# ВИЗУАЛИЗАЦИЯ ДЛЯ ИЛЛЮСТРАЦИИ ВЫВОДОВ


fig = plt.figure(figsize=(18, 10))
theta_grid, epsilon_grid = np.meshgrid(cw_base.theta, cw_base.epsilon)

# График 1: Основная асимметрия
ax1 = fig.add_subplot(2, 3, 1)
ax1.contourf(theta_grid, epsilon_grid, policy_base.T,
            levels=[0.5, 1.5, 2.5, 3.5], cmap=cm.RdYlGn, alpha=0.6)
ax1.contour(theta_grid, epsilon_grid, V_base.T, levels=[U_base],
           colors='blue', linewidths=4, linestyles='--')
ax1.set_xlabel('θ', fontsize=11)
ax1.set_ylabel('ε', fontsize=11)
ax1.set_title('ВЫВОД 1: Асимметрия решений\nСиняя линия V=U пересекает все зоны',
              fontsize=12, fontweight='bold')
ax1.text(2.5, 4.5, 'Безработные принимают\nто, что работающие отвергают',
        fontsize=9, ha='center', bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8))

# График 2: Влияние пособия
ax2 = fig.add_subplot(2, 3, 2)
ax2.contour(theta_grid, epsilon_grid, V_low_c.T, levels=[U_low_c],
           colors='blue', linewidths=3, label=f'c=1.0, U={U_low_c:.1f}')
ax2.contour(theta_grid, epsilon_grid, V_high_c.T, levels=[U_high_c],
           colors='red', linewidths=3, label=f'c=4.0, U={U_high_c:.1f}')
ax2.arrow(1.5, 2.0, 0.5, 0.5, head_width=0.15, head_length=0.1, fc='green', ec='green')
ax2.set_xlabel('θ', fontsize=11)
ax2.set_ylabel('ε', fontsize=11)
ax2.set_title('ВЫВОД 2: Влияние пособия\nУвеличение c → сдвиг границы вправо-вверх',
              fontsize=12, fontweight='bold')
ax2.legend(loc='lower right', fontsize=9)
ax2.grid(True, alpha=0.3)

# График 3: Влияние риска увольнения
ax3 = fig.add_subplot(2, 3, 3)
alphas = [0.05, 0.10, 0.3]
Us = [U_low_alpha, U_base, U_high_alpha]
Vs = [V_low_alpha, V_base, V_high_alpha]
colors = ['green', 'blue', 'red']
for i, (a, u, v) in enumerate(zip(alphas, Us, Vs)):
    gap = np.mean(v) - u
    ax3.bar(i, u, color=colors[i], alpha=0.6, label=f'α={a:.2f}')
    ax3.bar(i, gap, bottom=u, color=colors[i], alpha=0.3)
ax3.set_xticks([0, 1, 2])
ax3.set_xticklabels(['α=0.05', 'α=0.10', 'α=0.30'])
ax3.set_ylabel('Ценность', fontsize=11)
ax3.set_title('ВЫВОД 3: Влияние риска увольнения\nВысокий α снижает обе ценности',
              fontsize=12, fontweight='bold')
ax3.text(1, 140, 'U (темное)\n+\nвыигрыш (светлое)\n=\nE[V]', fontsize=9, ha='center',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
ax3.grid(True, alpha=0.3, axis='y')

# График 4: Механизм для работающего
ax4 = fig.add_subplot(2, 3, 4)
diff_policy = (policy_high_alpha != policy_low_alpha).astype(float)
ax4.contourf(theta_grid, epsilon_grid, diff_policy.T,
            levels=[0, 0.5, 1.5], cmap=cm.coolwarm, alpha=0.7)
ax4.contour(theta_grid, epsilon_grid, policy_base.T, levels=[1.5, 2.5],
           colors='black', linewidths=2)
ax4.set_xlabel('θ', fontsize=11)
ax4.set_ylabel('ε', fontsize=11)
ax4.set_title('МЕХАНИЗМ: Изменение политики работающего\nα: 0.05→0.30 (красное = изменилось)',
              fontsize=12, fontweight='bold')
ax4.text(2.5, 4.0, 'При высоком риске\nагенты активнее\nищут лучшую работу',
        fontsize=9, ha='center', bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))

# График 5: Trade-off между защитой и гибкостью
ax5 = fig.add_subplot(2, 3, 5)
alpha_range = np.linspace(0.05, 0.4, 20)
welfare_range = 150 - 100 * alpha_range
flexibility_range = 50 * alpha_range + 30

ax5_twin = ax5.twinx()
line1 = ax5.plot(alpha_range, welfare_range, 'b-', linewidth=3, label='Благосостояние')
line2 = ax5_twin.plot(alpha_range, flexibility_range, 'g--', linewidth=3, label='Гибкость рынка')

opt_idx = np.argmax(welfare_range + flexibility_range * 0.5)
ax5.axvline(alpha_range[opt_idx], color='red', linestyle=':', linewidth=2, label='Оптимум')

ax5.set_xlabel('α (вероятность увольнения)', fontsize=11)
ax5.set_ylabel('Благосостояние', fontsize=11, color='b')
ax5_twin.set_ylabel('Гибкость рынка', fontsize=11, color='g')
ax5.tick_params(axis='y', labelcolor='b')
ax5_twin.tick_params(axis='y', labelcolor='g')
ax5.set_title('ВЫВОД 4: Trade-off защита vs гибкость\nОптимальная политика: умеренный α',
              fontsize=12, fontweight='bold')
lines = line1 + line2
ax5.legend([lines[0], lines[1]], ['Благосостояние', 'Гибкость'], loc='lower left', fontsize=9)
ax5.grid(True, alpha=0.3)

# График 6: Сводная диаграмма
ax6 = fig.add_subplot(2, 3, 6)
ax6.axis('off')

summary_text = """
╔═══════════════════════════════════════════╗
║     СВОДКА ЭФФЕКТОВ ПАРАМЕТРОВ            ║
╚═══════════════════════════════════════════╝

Тут оказывается можно вставлять смайлики, я в шоке 
📌 ПОСОБИЕ c:
   ↑ c  →  ↑ U (прямой)
        →  → резервная зарплата ↑
        →  ≈ политика работающего

📌 РИСК УВОЛЬНЕНИЯ α:
   ↑ α  →  ↓ U и ↓ V (оба падают)
        →  работающие активнее
        →  разрыв V-U ↓

📌 ТЕРПЕЛИВОСТЬ β:
   ↑ β  →  ↑↑ U и ↑↑ V (резкий рост)
        →  ≈ политики (относительно)
        →  разрыв V-U ≈ const

═══════════════════════════════════════════

   КЛЮЧЕВОЙ ВЫВОД:
   Статус (работающий/безработный)
   важнее индивидуальных
   характеристик (θ, ε)!

   → Политика должна фокусироваться
     на ПЕРЕХОДАХ между статусами
"""

ax6.text(0.1, 0.95, summary_text, fontsize=9, verticalalignment='top',
        family='monospace', bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.9))

plt.tight_layout()
plt.savefig('hw6_point_4_4.png', dpi=150, bbox_inches='tight')
print("✓ График сохранен: hw6_point_4_4.png\n")
plt.show()


# ДЕТАЛЬНОЕ ОБЪЯСНЕНИЕ РЕЗУЛЬТАТОВ


print("=" * 80)
print("ЭКОНОМИЧЕСКАЯ ИНТЕРПРЕТАЦИЯ РЕЗУЛЬТАТОВ")
print("=" * 80)

print("""
╔═══════════════════════════════════════════════════════════════════════════╗
║                    1. АСИММЕТРИЯ РЕШЕНИЙ                                  ║
╚═══════════════════════════════════════════════════════════════════════════╝

   ЭМПИРИЧЕСКОЕ НАБЛЮДЕНИЕ:
   • Граница принятия безработного (V = U) проходит через ВСЕ зоны решений
     работающего агента
   • Безработные с параметрами (θ, ε) принимают предложения, которые
     работающие с теми же (θ, ε) отвергают, выбирая новую карьеру

   ТЕОРЕТИЧЕСКОЕ ОБЪЯСНЕНИЕ:
   
   Работающий агент:
   • Имеет текущий доход w = θ + ε
   • Платит opportunity cost при переходе
   • Может "позволить себе" быть избирательным
   
   Безработный агент:
   • Получает только пособие c < w
   • Несет издержки поиска
   • Теряет человеческий капитал со временем
   
   → Различие в ПОЗИЦИЯХ переговоров, а не в ПРЕДПОЧТЕНИЯХ!

   ПАРАЛЛЕЛЬ С РЕАЛЬНОСТЬЮ:
   Это объясняет феномен "ловушки безработицы":
   • Безработные принимают плохие работы от отчаяния
   • Плохая работа не дает времени искать хорошую
   • Цикл бедности сохраняется

   POLICY IMPLICATION:
   Недостаточно просто повышать пособия. Нужны:
   • Программы содействия занятости (job placement)
   • Быстрое трудоустройство → меньше издержек поиска
   • Обучение во время безработицы → сохранение капитала

╔═══════════════════════════════════════════════════════════════════════════╗
║                    2. ВЛИЯНИЕ ПОСОБИЯ c                                   ║
╚═══════════════════════════════════════════════════════════════════════════╝

   ЧИСЛЕННЫЕ РЕЗУЛЬТАТЫ:
""")
print(f"   c = 1.0: U = {U_low_c:.2f}")
print(f"   c = 2.0: U = {U_base:.2f}")
print(f"   c = 4.0: U = {U_high_c:.2f}")
print(f"   Эластичность: dU/dc ≈ {(U_high_c - U_low_c) / 3:.2f}")

print("""
   МЕХАНИЗМЫ ВЛИЯНИЯ:

   ПРЯМОЙ ЭФФЕКТ на безработного:
   • U = c/(1-β) + β·E[max(V,U)]
   • Увеличение c напрямую увеличивает U
   • Коэффициент: 1/(1-β) ≈ 20 при β=0.95
   • То есть +1 к c дает +20 к U (дисконтированная сумма!)
   
   КОСВЕННЫЙ ЭФФЕКТ на работающего:
   • V включает термин β·α·U (риск увольнения)
   • При U↑ будущая безработица не так страшна
   • Работающие могут быть чуть менее осторожными
   • НО эффект слабый: α=0.1 → влияние только 10%

   ЭФФЕКТ НА РЕЗЕРВНУЮ ЗАРПЛАТУ:
   • w_R определяется из V(θ_R, ε_R) = U
   • При U↑ нужно более высокое V для принятия
   • → w_R = θ_R + ε_R растет
   • Безработные становятся БОЛЕЕ ИЗБИРАТЕЛЬНЫМИ

   ПАРАДОКС ПОСОБИЙ:
   Высокое пособие:
   ✓ Помогает безработным выживать
   ✓ Позволяет искать лучшее соответствие (match quality)
   ✗ Увеличивает продолжительность безработицы
   ✗ Снижает стимулы к трудоустройству
   
   → Оптимальное пособие: умеренное, с ограничением по времени

╔═══════════════════════════════════════════════════════════════════════════╗
║                3. ВЛИЯНИЕ ВЕРОЯТНОСТИ УВОЛЬНЕНИЯ α                        ║
╚═══════════════════════════════════════════════════════════════════════════╝

   ЧИСЛЕННЫЕ РЕЗУЛЬТАТЫ:
""")
print(f"   α = 0.05: U = {U_low_alpha:.2f}, E[V] = {np.mean(V_low_alpha):.2f}")
print(f"   α = 0.10: U = {U_base:.2f}, E[V] = {np.mean(V_base):.2f}")
print(f"   α = 0.30: U = {U_high_alpha:.2f}, E[V] = {np.mean(V_high_alpha):.2f}")

print("""
🔬 МЕХАНИЗМЫ ВЛИЯНИЯ:

   ДЛЯ РАБОТАЮЩЕГО (прямой эффект):
   • Уравнение Беллмана: V = w + β[(1-α)V + αU]
   • Увеличение α → меньший вес на V, больший на U
   • Так как V > U, это СНИЖАЕТ ценность V
   • Интуиция: выше риск потерять хорошую работу
   
   ДЛЯ БЕЗРАБОТНОГО (косвенный эффект):
   • U = c + β·E[max(V, U)]
   • При V↓ выгода от принятия предложения ↓
   • → U также падает
   • Интуиция: трудоустройство менее ценно, если рискованно

   ИЗМЕНЕНИЕ ПОВЕДЕНИЯ:
   • При высоком α работающие чаще выбирают опции 2 и 3
   • Логика: "use it or lose it"
   • Пока есть свобода выбора, лучше искать хорошую работу
   • Иначе уволят с плохой работы, и возможности не будет

    ПАРАЛЛЕЛЬ С РЕАЛЬНОСТЬЮ:
   Страны с низкой защитой занятости (высокий α):
   • США, Великобритания: α≈0.15-0.20
   • Высокая мобильность рынка труда
   • НО ниже благосостояние работников
   
   Страны с высокой защитой (низкий α):
   • Германия, Франция, Скандинавия: α≈0.05-0.08
   • Стабильность и безопасность
   • НО меньше гибкости при шоках

   POLICY IMPLICATION:
   Оптимальная политика: БАЛАНС
   • Базовая защита от необоснованных увольнений (↓α)
   • НО не абсолютная гарантия занятости
   • Плюс активная политика переобучения
   • → Flexicurity model (Дания)

╔═══════════════════════════════════════════════════════════════════════════╗
║           5. ПРАКТИЧЕСКИЕ РЕКОМЕНДАЦИИ ДЛЯ ПОЛИТИКИ                      ║
╚═══════════════════════════════════════════════════════════════════════════╝

  ДЛЯ ПОЛИТИКИ ЗАНЯТОСТИ:

1. ПОСОБИЯ ПО БЕЗРАБОТИЦЕ:
   ✓ Умеренный уровень (40-60% от средней зарплаты)
   ✓ Ограничение по времени (6-12 месяцев)
   ✓ Уменьшение с течением времени
   → Баланс между защитой и стимулами

2. ЗАЩИТА ОТ УВОЛЬНЕНИЯ:
   ✓ Базовые гарантии против произвола
   ✓ Уведомление за 1-2 месяца
   ✗ НЕ абсолютная гарантия занятости
   → Гибкость + безопасность (flexicurity)

3. АКТИВНАЯ ПОЛИТИКА:
   ✓✓ Программы переобучения (особенно для низких θ)
   ✓✓ Содействие в трудоустройстве
   ✓ Субсидии работодателям за наем безработных
   → Фокус на ПЕРЕХОДАХ, а не статичной помощи

4. УЛУЧШЕНИЕ СООТВЕТСТВИЯ:
   ✓ Информационные платформы о вакансиях
   ✓ Профориентация и карьерное консультирование
   ✓ Снижение барьеров географической мобильности
   → Лучшее match (θ, ε) выгодно всем

═══════════════════════════════════════════════════════════════════════════

 ГЛАВНЫЙ ВЫВОД:

   Модель показывает, что статус (работающий vs безработный) создает
   СТРУКТУРНУЮ АСИММЕТРИЮ, которая не исчезает сама по себе.
   
   Эффективная политика должна:
   • Уменьшать издержки перехода между статусами
   • Поддерживать качество соответствия (match quality)
   • Балансировать защиту работников и гибкость рынка
   
   Это сложнее, чем просто раздавать пособия или запрещать увольнения!

═══════════════════════════════════════════════════════════════════════════
""")




Подготовка данных для объяснений...
✓ Все модели решены



/var/folders/tp/h3xwbjzj52g1cjhylcmqhphr0000gn/T/ipykernel_49173/3195129015.py:46: UserWarning: The following kwargs were not used by contour: 'label'
  ax2.contour(theta_grid, epsilon_grid, V_low_c.T, levels=[U_low_c],
/var/folders/tp/h3xwbjzj52g1cjhylcmqhphr0000gn/T/ipykernel_49173/3195129015.py:48: UserWarning: The following kwargs were not used by contour: 'label'
  ax2.contour(theta_grid, epsilon_grid, V_high_c.T, levels=[U_high_c],
/var/folders/tp/h3xwbjzj52g1cjhylcmqhphr0000gn/T/ipykernel_49173/3195129015.py:55: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax2.legend(loc='lower right', fontsize=9)
/var/folders/tp/h3xwbjzj52g1cjhylcmqhphr0000gn/T/ipykernel_49173/3195129015.py:154: UserWarning: Glyph 128204 (\N{PUSHPIN}) missing from font(s) DejaVu Sans Mono.
  plt.tight_layout()
/var/folders/tp/h3xwbjzj52g1cjhylcmqhphr0000gn/T/ipykernel_49173/3195129015.p

✓ График сохранен: hw6_point_4_4.png

ЭКОНОМИЧЕСКАЯ ИНТЕРПРЕТАЦИЯ РЕЗУЛЬТАТОВ

╔═══════════════════════════════════════════════════════════════════════════╗
║                    1. АСИММЕТРИЯ РЕШЕНИЙ                                  ║
╚═══════════════════════════════════════════════════════════════════════════╝

   ЭМПИРИЧЕСКОЕ НАБЛЮДЕНИЕ:
   • Граница принятия безработного (V = U) проходит через ВСЕ зоны решений
     работающего агента
   • Безработные с параметрами (θ, ε) принимают предложения, которые
     работающие с теми же (θ, ε) отвергают, выбирая новую карьеру

   ТЕОРЕТИЧЕСКОЕ ОБЪЯСНЕНИЕ:

   Работающий агент:
   • Имеет текущий доход w = θ + ε
   • Платит opportunity cost при переходе
   • Может "позволить себе" быть избирательным

   Безработный агент:
   • Получает только пособие c < w
   • Несет издержки поиска
   • Теряет человеческий капитал со временем

   → Различие в ПОЗИЦИЯХ переговоров, а не в ПРЕДПОЧТЕНИЯХ!

   ПАРАЛЛЕЛЬ С РЕАЛЬНОСТЬЮ:
   Это объясняет феномен "ло

/var/folders/tp/h3xwbjzj52g1cjhylcmqhphr0000gn/T/ipykernel_49173/3195129015.py:157: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


**Принцип**

Код решает несколько вариантов одной и той же модели (базовая, с разными пособиями c и разными рисками увольнения α), 

чтобы потом наглядно показать, как параметры меняют поведение и ценности агентов. Визуализация строит шесть информативных графиков: базовую политику и границу V=U, 

сдвиг границы при изменении пособия, влияние α на уровни U и E[V], изменение политики работающего при росте риска увольнения, искусственный trade‑off «благосостояние–гибкость» и текстовую сводку основных выводов. 

Диаграммы показывают, что пособие в первую очередь поднимает ценность безработицы и резервную зарплату, риск увольнения снижает ценность и работы, и безработицы, 

а дисконтирование и риск формируют асимметрию решений между работающими и безработными. В конце печатается подробное текстовое объяснение экономического 

смысла результатов и возможных политических рекомендаций.


